In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
pd.options.display.float_format = '{:,.0f}'.format

In [3]:
df = pd.read_csv("Zephyr_data.csv", encoding = 'mac_roman')


/Users/nickbatarilo/anaconda3/envs/Young_proj_1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10,11,29,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
cols = [10,11,29,32,27,28,29,30,31,26,33,25,24,23,22,21,20,19,17,15,14,13,12,11,10,9,8,7,1,3,5]
df.drop(df.columns[cols],axis=1,inplace=True)

In [5]:
df['year']=pd.to_datetime(df['Last deal status date'], format='%m/%d/%y').dt.year.fillna(0.0).astype(int)

In [6]:
df.drop(df.columns[6],axis=1,inplace=True)
df.drop(df.columns[0],axis=1,inplace=True)

In [7]:
df.replace("n.a.", np.nan, inplace=True)
df.replace(0, np.nan, inplace=True)
df= df.replace("-",np.nan)


In [8]:
isrumor = (df['Deal status'] == "Rumour - Withdrawn") | (df['Deal status'] == "Rumour") | (df['Deal status'] == "Rumour - Analyst Speculation") | (df['Deal status'] == "Unconditional") | (df['Deal status'] == "Postponed") | (df['Deal status'] == "Rumour - informal offer/non-binding")          


In [9]:
df.head()

SyntaxError: invalid syntax (<ipython-input-9-71f11347ebfc>, line 1)

In [ ]:
isnan = df['Deal status'].isnull()
rumorbelownan = isnan.shift(-1) & isrumor
notisnullbelownull = isnan.shift(1) & ~isnan
rowtodelete = []

In [ ]:
for i, v in rumorbelownan.items():
    if v == True:
        rowtodelete.append(i)
        for j, w in notisnullbelownull.iloc[i+1:].items():
            if w == False:
                rowtodelete.append(j)
            elif w == True:
                break

In [ ]:
df.drop(rowtodelete, axis=0, inplace=True)

In [ ]:

df = df[~((df['Deal status'] == "Rumour - Withdrawn") | (df['Deal status'] == "Rumour") | (df['Deal status'] == "Rumour - Analyst Speculation") | (df['Deal status'] == "Unconditional") | (df['Deal status'] == "Postponed") | (df['Deal status'] == "Rumour - informal offer/non-binding"))]      


In [ ]:
df.reset_index(inplace=True, level=None, drop=True)

In [ ]:
isrumor2 = (df['Deal status'] == "Rumour - Expired")
isnan2 = df['Deal status'].isnull()
rumorbelownan2 = isnan2.shift(-1) & isrumor2
notisnullbelownull2 = isnan2.shift(1) & ~isnan2
rowtodelete2 = []
for i, v in rumorbelownan2.items():
    if v == True:
        rowtodelete2.append(i)
        for j, w in notisnullbelownull2.iloc[i+1:].items():
            if w == False:
                rowtodelete2.append(j)
            elif w == True:
                break

In [ ]:
df.drop(rowtodelete2, axis=0, inplace=True)
df = df[~(df['Deal status'] == "Rumour - Expired")]

In [ ]:
df.reset_index(inplace=True, level=None, drop=True)

In [ ]:
# Might seem useless but important for functional program
df = df.copy()

In [ ]:
#Crucial that this line is after the nan count but before the data filling
df.dropna(how='all', inplace=True)

In [ ]:
df["Acquiror country code"].fillna(method = 'ffill', inplace=True) 
df["Deal status"].fillna(method = 'ffill', inplace=True)
df['year'].fillna(method = 'ffill', inplace = True)

In [ ]:
df['Acquiror GUO ISO country code'].fillna(df['Acquiror country code'], inplace=True)
df['Target GUO ISO country code'].fillna(df['Target country code'], inplace=True)
df['Target GUO ISO country code'].fillna(method = 'ffill', inplace = True)

In [ ]:
df.drop(df.columns[[0,1]],axis=1,inplace=True)

In [ ]:
deals = ['Announced', 'Completed', 'Completed Assumed', 'Pending', 'Withdrawn']
deal_type = np.array(deals)
target_country = df['Target GUO ISO country code'].unique().tolist()
acquiror_country = df['Acquiror GUO ISO country code'].unique().tolist()
years = [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 
         2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
year_range = np.array(years)

In [ ]:
df.replace(to_replace = 'Pending - awaiting regulatory approval', value = 'Pending', inplace=True)
df.replace(to_replace = 'Pending - awaiting shareholder approval', value = 'Pending', inplace=True)

In [ ]:
f = open('output.csv', 'w+')
s = "year, ccode1, ccode2, deal_announce, deal_complete, deal_complete_assumed, deal_pend, deal_withdrawn, total"
f.write(s+"\n")

In [ ]:
seperator = ', '
for a in acquiror_country:
    for t in target_country:
        for y in years:
            total = 0
            f.write(str(y) + seperator)
            f.write(a + seperator)
            f.write(t + seperator)
            for d in deals:
                temp_df = (df['Deal status'] == d) & (df['Acquiror GUO ISO country code'] == a) & (df['Target GUO ISO country code'] == t) & (df['year'] == y)
                count = np.count_nonzero(temp_df)
                f.write(str(count) + seperator)
                total += count
                
            f.write(str(total) + "\n")                
                

In [ ]:
f.close()